# Default to look for RSS feed
First brought to attention when realized that Pfizer's press release page is .aspx and the links cannot be pulled from a simple requests.get command. Still looking into how to scrape that directly but in the meantime, RSS is a reliable way (if available) to check for updates (often with even more metadata). 

In [1]:
%load_ext autoreload
%autoreload 2

import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

import sys
sys.path.append("../python/")

import medwatch as mw

/Users/hugh/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [46]:
def is_scrapable(url):
    # Checks if url is scrapable. For now, we know that .aspx is not
    if url[-4:]=='aspx':
        return False
    
    
def search4rss(url):
    r = requests.get(url)
    page = r.text
    page = ' '.join(page.split())
    soup = BeautifulSoup(page, "html.parser")
    
    anchors = mw.get_anchors(soup)
    
    rss_anchors, keywords = mw.check_for_keywords(anchors, keywords=['rss'], keywords_ignore=[])
    
    return rss_anchors

In [47]:
url = 'https://investors.pfizer.com/investor-news/default.aspx'

print(is_scrapable(url))

rss_anchors = search4rss(url)
print(rss_anchors)
href, _ = mw.parse_anchors(rss_anchors)
href = str(href[0])
rss_url = urljoin(url, str(href))
print(rss_url)

False
Match because of following keywords: ['rss']
[<a class="RssLinkTop" href="/rss/pressrelease.aspx" target="_blank"> <i aria-hidden="true" class="q4i-icon-rss"></i> <span class="sr-only">Press Release Feed (opens in new window)</span> </a>]
https://investors.pfizer.com/rss/pressrelease.aspx


In [44]:
r = requests.get(rss_url)
# print(r.text)
soup = BeautifulSoup(r.text, 'xml') # Make sure it's xml and not html.parser
links = soup.find_all('link')

for link in links:
    print(link.text)

https://investors.pfizer.com/
https://investors.pfizer.com/investor-news/press-release-details/2020/Pfizer-and-BioNTech-Share-Positive-Early-Data-on-Lead-mRNA-Vaccine-Candidate-BNT162b2-Against-COVID-19/default.aspx
https://investors.pfizer.com/investor-news/press-release-details/2020/Pfizer-Announces-Agreement-with-Gilead-to-Manufacture-Remdesivir-for-Treatment-of-COVID-19/default.aspx
https://investors.pfizer.com/investor-news/press-release-details/2020/Pfizer-Announces-Times-for-Rescheduled-Virtual-Investor-Day-on-September-14-and-September-15-2020/default.aspx
https://investors.pfizer.com/investor-news/press-release-details/2020/LORBRENA-lorlatinib-Significantly-Improves-Progression-Free-Survival-in-First-Line-ALK-Positive-Lung-Cancer/default.aspx
https://investors.pfizer.com/investor-news/press-release-details/2020/Pfizer-and-BioNTech-to-Supply-Canada-with-their-BNT162-mRNA--Based-Vaccine-Candidate/default.aspx
https://investors.pfizer.com/investor-news/press-release-details/2020/